<a href="https://colab.research.google.com/github/thainhf/new-taggerbot-training-model/blob/main/Clean_data_taggerbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gspread as gs
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from google.auth import default

In [30]:
import nltk

In [4]:
df_defin = pd.read_excel('/content/drive/MyDrive/Taggerbot/Tag_Definition.xlsx', header=0)
df_train_IPST = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_IPST_20Expert.xlsx', header=0)
df_train_Lump100 = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Lumplymat_100Doc_10Expert.xlsx', header=0)
df_train_Lumpkk = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Lumplymat_KK_Conf.xlsx', header=0)
df_train_Qinfo = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Qinfo2_15Expert.xlsx', header=0)
df_train_school = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_School_Management.xlsx', header=0)


In [10]:
df_train_IPST = df_train_IPST.rename(columns={'name.1':'tag'})
df_train_Qinfo = df_train_Qinfo.rename(columns={'name.1':'tag'})

In [13]:
df_train_Lump100 = df_train_Lump100.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})
df_train_Lumpkk = df_train_Lumpkk.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})
df_train_school = df_train_school.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})

In [16]:
df_train_main = pd.concat([df_train_IPST[['content','tag']], df_train_Lump100[['content','tag']], df_train_Lumpkk[['content','tag']],
                           df_train_Qinfo[['content','tag']], df_train_school[['content','tag']]], axis=0)
df_train_main

,content,tag
0,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,Analyse
1,จากสถานการณ์ที่กำหนดให้ นักเรียนตอบคำถามลงในใบ...,Analyse
2,นักเรียนแต่ละคนภายในกลุ่มนำข้อมูลที่ได้ศึกษาค้...,Analyse
3,การเคลื่อนที่ของเม็ดโฟมผูกด้วยด้ายเล็ก ที่มีคว...,Analyse
4,สึนามิแตกต่างจากคลื่นธรรมดาอย่างไร,Analyse
...,...,...
1624,มากไปกว่านั้น เด็กๆ ยังคิดต่อได้ว่า ควรมีสาระ...,Critical Thinking
1625,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,Information Literacy Skill
1626,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,ICT Literacy
1627,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,Media Literacy


In [18]:
set(df_train_main['tag'])

{'Accountability',
 'Analyse',
 'Apply',
 'Collaboration Skill',
 'Communication Skill',
 'Compassion',
 'Creativity and Innovation',
 'Critical Thinking',
 'Critical Thinking and Problem Solving',
 'Decision Skill',
 'Evaluate',
 'Flexibility and Adaptability',
 'Flexibility and adaptability',
 'ICT  Literacy',
 'ICT Literacy',
 'Information Lieteracy Skill',
 'Information Literacy Skill',
 'Initiative and Self-Direction',
 'Initiative and self-direction',
 'Leadership and responsibity',
 'Media Literacy',
 'Media Literacy Skills',
 'Productivity',
 'Productivity and Accountability',
 'Remember',
 'Social and Cross-Cultural Skills',
 'Understanding'}

In [58]:
df_train_main['content'] = df_train_main['content'].str.replace('[\\<>\;\[\]\:\-*\*\$\,\.\?!"\'\^\()\&\/\{\}\|\£\«\»\’\“\”\•\■\™\~]\s*', '', regex=True)
df_train_main = df_train_main.reset_index(drop=True)
df_train_main

,content,tag
0,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,Analyse
1,จากสถานการณ์ที่กำหนดให้ นักเรียนตอบคำถามลงในใบ...,Analyse
2,นักเรียนแต่ละคนภายในกลุ่มนำข้อมูลที่ได้ศึกษาค้...,Analyse
3,การเคลื่อนที่ของเม็ดโฟมผูกด้วยด้ายเล็ก ที่มีคว...,Analyse
4,สึนามิแตกต่างจากคลื่นธรรมดาอย่างไร,Analyse
...,...,...
34707,มากไปกว่านั้น เด็กๆ ยังคิดต่อได้ว่า ควรมีสาระ...,Critical Thinking
34708,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,Information Literacy Skill
34709,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,ICT Literacy
34710,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,Media Literacy


In [69]:
df = (df_train_main.groupby(['content'])
      .agg({'tag': lambda x: ",".join(x)})
      .reset_index())
df_contain = df[df['tag'].str.contains(',')]
df_contain

,content,tag
1,ส่วนร่ว...,"Collaboration Skill,Flexibility and adaptability"
5,นอกจากนี้ครูยังจัดที่นั...,"Accountability,Flexibility and Adaptability"
6,ผลที่เกิดขึ้นกับเด็กทำใ...,"Communication Skill,Collaboration Skill"
9,การที่ได้นำเป้าหมายประจำสัป...,"Accountability,Initiative and Self-Direction"
12,จากเหตุการณ์ครั้งนี้ทำให้รู้ว่...,"Communication Skill,Collaboration Skill"
...,...,...
5947,๔งานการ์ตูนมีการบรรยายความรู้แทรก,"Critical Thinking,Communication Skill"
5951,๕งานโฟกัสมีการเปรียบเทียบใบพัดของพัดลมกับเฮลิค...,"Critical Thinking,Communication Skill"
5952,๕นักเรียนเป็นนักสังเกต เข้าหาความรู้ได้เอง,"Productivity,Accountability,Initiative and Sel..."
5953,๕ประเด็นที่จะนำไปพัฒนาต่อ คือ การฝึกตั้งประเด็...,"Creativity and Innovation,Initiative and Self-..."


In [68]:
df_contain.iloc[0,0]

'                                       ส่วนร่วมในการดูแลรักษาสิ่งแวดล้อม'